# Eval: Other model with statement-result set

This notebook outlines the steps undertook to evaluate various other models out of the box with the test set under a IR set-up.

With 512 chunk size.

# Import libraries

In [2]:
#%pip install nltk -q
#%pip install sentence_transformers -q

import numpy as np
import pandas as pd

import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')

from sentence_transformers import SentenceTransformer, util

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# Global variable

In [3]:
EVALS = {}
MAX_WORDS = 512

# Helper functions

In [4]:
def split_text_into_chunks(text, tag="", max_words=MAX_WORDS, overlap_sentences=3):
    if tag == "query":
      text = "search_query: " + text
    elif tag == "opinion":
      text = "search_document: " + text
    else:
      text = text

    sentences = sent_tokenize(text)  # Tokenize into sentences
    chunks = []
    start_idx = 0  # Start index of the current chunk

    while start_idx < len(sentences):
        current_chunk = []
        current_word_count = 0
        idx = start_idx

        # Build a chunk until max_words is reached
        while idx < len(sentences) and current_word_count + len(sentences[idx].split()) <= max_words:
            current_chunk.append(sentences[idx])
            current_word_count += len(sentences[idx].split())
            idx += 1  # Move to next sentence

        if current_chunk:
            chunks.append(" ".join(current_chunk))

        # Stop if the last chunk reaches the end of the text
        if idx >= len(sentences):
            break

        # Move start index forward but keep overlap
        start_idx = max(idx - overlap_sentences, start_idx + 1)

    return chunks


def create_embeddings(df, model, text_column, embedding_column, tag):
    df[embedding_column] = None

    for idx, text in enumerate(df[text_column]):
        chunks = split_text_into_chunks(text, tag)  # Chunk the text
        chunk_embeddings = np.array(model.encode(chunks))  # Encode all chunks (shape: [num_chunks, embedding_dim])
        df.at[idx, embedding_column] = chunk_embeddings

    return df


def evaluate_retrieval(query_emb, doc_embs, relevant_idx, top_k=5):
    # Flatten the document chunks and keep track of the document they belong to
    all_chunk_embeddings = []
    doc_chunk_mapping = []  # Mapping from chunk index to document index
    for doc_idx, doc_chunks in enumerate(doc_embs):
        for chunk_emb in doc_chunks:
            all_chunk_embeddings.append(chunk_emb)
            doc_chunk_mapping.append(doc_idx)  # Store the document index for each chunk

    # Convert to numpy array for efficient computation
    all_chunk_embeddings = np.array(all_chunk_embeddings)  # Shape: [total_chunks, 768]

    # Compute cosine similarity between query and all document chunks
    similarities = np.dot(all_chunk_embeddings, query_emb.T)  # Shape: [total_chunks,]

    # Get indices of top-k most similar chunks
    top_k_chunk_indices = np.argsort(similarities.flatten())[::-1][:top_k]

    # Find the documents corresponding to the top-k chunks
    retrieved_docs = [doc_chunk_mapping[idx] for idx in top_k_chunk_indices]

    # Count the number of times the relevant document appears in the top-k retrieved documents
    relevant_document_hits = sum(1 for doc_idx in retrieved_docs if doc_idx == relevant_idx)

    # Compute rank (based on first appearance of the relevant document)
    rank = None
    for idx, doc_idx in enumerate(retrieved_docs):
        if doc_idx == relevant_idx:
            rank = idx + 1
            break

    # Compute metrics
    is_hit = relevant_document_hits > 0  # Hit if any of the top-k chunks belong to the relevant document
    mrr = 0 if rank is None else 1 / rank

    return {
        "is_hit": is_hit,
        "mrr": mrr,
        "retrieved": retrieved_docs,
        "expected": relevant_idx
    }


def evaluate_model(df, model, query_column, query_emb_column, opinion_column, opinion_emb_column, tags=["",""], top_k=5):
    # Step 1: Create embeddings for queries and opinions
    df = create_embeddings(df, model, query_column, query_emb_column, tag=tags[0])
    df = create_embeddings(df, model, opinion_column, opinion_emb_column, tag=tags[1])

    # Step 2: Evaluate retrieval for each query
    results_dict = {}
    for idx, query_row in df.iterrows():
        opinion_id = query_row["opinion_id"]
        query_emb = query_row[query_emb_column]
        doc_embs = df[opinion_emb_column]
        results_dict[opinion_id] = evaluate_retrieval(query_emb, doc_embs, idx, top_k=top_k)

    # Step 3: Convert results to DataFrame
    results_df = pd.DataFrame.from_dict(results_dict, orient='index').reset_index().rename(columns={'index': 'opinion_id'})

    # Step 4: Merge results with the original DataFrame
    final_df = df.merge(results_df, how="left", on="opinion_id")

    # Step 5: Calculate evaluation metrics (Hit Rate and MRR)
    model_name = model.name
    EVALS[model_name] = {}
    EVALS[model_name]["hit_rate"] = final_df["is_hit"].mean()
    EVALS[model_name]["mrr"] = final_df["mrr"].mean()

    return final_df

# Load the data

In [5]:
df = pd.read_csv("outputs/3.test.csv")
df = df[["opinion_id", "opinion_word_count", "opinion", "relevant_query_stmt"]]
df.head()

,opinion_id,opinion_word_count,opinion,relevant_query_stmt
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",contractual obligations in maritime law
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,negligent retention employee sexual harassment
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,ineffective assistance of appellate counsel in...
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,Roger Fincher's appeal of order denying rule 3...
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,Appeal from a conviction for aggravated sexual...


# Evaluate model: Alibaba-NLP/gte-large-en-v1.5

In [6]:
%%time

model_name = "Alibaba-NLP/gte-large-en-v1.5"
model = SentenceTransformer(model_name, trust_remote_code=True)
model.name = model_name
result1 = evaluate_model(df, model, "relevant_query_stmt", "query_embeddings", "opinion", "opinion_embeddings")

result1.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

CPU times: user 6min 14s, sys: 5.82 s, total: 6min 20s
Wall time: 7min 1s


,opinion_id,opinion_word_count,opinion,relevant_query_stmt,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",contractual obligations in maritime law,"[[-0.6487742, -0.5020727, 0.21850303, 0.624837...","[[-0.3680863, 0.31216347, -0.02005817, 0.27055...",True,1.0,"[0, 303, 90, 269, 387]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,negligent retention employee sexual harassment,"[[-0.19732316, -0.0027027293, -0.8593739, -0.0...","[[0.42036155, 0.18966155, -0.969184, -0.717158...",True,1.0,"[1, 1, 1, 112, 112]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,ineffective assistance of appellate counsel in...,"[[0.092208534, 0.16966999, -0.12987915, 0.4525...","[[0.31887192, 0.059133492, 0.92121917, 0.38830...",False,0.0,"[165, 425, 124, 165, 375]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,Roger Fincher's appeal of order denying rule 3...,"[[-0.8378395, -0.12694854, -0.55889356, -0.134...","[[-0.22347826, -0.09282645, 0.70723313, 0.3846...",True,1.0,"[3, 16, 172, 279, 357]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,Appeal from a conviction for aggravated sexual...,"[[0.008925746, -0.24891691, -0.2600596, 0.0324...","[[0.83855593, 0.20392627, -1.2022392, -0.06087...",False,0.0,"[297, 58, 297, 297, 297]",4


In [7]:
result1.to_csv("4b.eval/IR-512/3.result1.csv", index=False)
len(result1)

450

In [8]:
EVALS

{'Alibaba-NLP/gte-large-en-v1.5': {'hit_rate': 0.8022222222222222,
  'mrr': 0.7117407407407407}}

# Evaluate model: thenlper/gte-large

In [9]:
%%time

model_name = "thenlper/gte-large"
model = SentenceTransformer(model_name)
model.name = model_name
result2 = evaluate_model(df, model, "relevant_query_stmt", "query_embeddings", "opinion", "opinion_embeddings")

result2.head()

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

CPU times: user 3min 3s, sys: 3.91 s, total: 3min 7s
Wall time: 3min 16s


,opinion_id,opinion_word_count,opinion,relevant_query_stmt,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",contractual obligations in maritime law,"[[0.001698609, 0.004124036, -0.020399854, -0.0...","[[-0.012209993, -0.020404605, -0.010719932, 0....",True,0.5,"[381, 0, 409, 381, 381]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,negligent retention employee sexual harassment,"[[-0.0045413184, -0.003497314, -0.015070536, 0...","[[-0.017438138, -0.023467613, -0.0195696, 0.01...",True,1.0,"[1, 1, 1, 299, 299]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,ineffective assistance of appellate counsel in...,"[[-0.012722618, 0.0044750017, -0.0037620466, 0...","[[-0.026284987, -0.012898638, -0.0124339145, 0...",False,0.0,"[425, 165, 95, 124, 78]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,Roger Fincher's appeal of order denying rule 3...,"[[0.011759969, -0.028919654, -0.004358438, 0.0...","[[-0.0037303022, -0.014129647, -0.0053730686, ...",True,1.0,"[3, 154, 32, 55, 334]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,Appeal from a conviction for aggravated sexual...,"[[-0.021319946, -0.0128439525, -0.015868478, 0...","[[-0.029757459, 0.0010514171, 0.006418267, 0.0...",False,0.0,"[295, 20, 281, 297, 329]",4


In [10]:
result2.to_csv("4b.eval/IR-512/3.result2.csv", index=False)
len(result2)

450

In [11]:
EVALS

{'Alibaba-NLP/gte-large-en-v1.5': {'hit_rate': 0.8022222222222222,
  'mrr': 0.7117407407407407},
 'thenlper/gte-large': {'hit_rate': 0.8088888888888889,
  'mrr': 0.6997037037037037}}

# Evaluate model: nomic-ai/nomic-embed-text-v1

In [14]:
%%time

model_name = "nomic-ai/nomic-embed-text-v1"
model = SentenceTransformer(model_name, trust_remote_code=True)
model.name = model_name
result3 = evaluate_model(df, model, "relevant_query_stmt", "query_embeddings", "opinion", "opinion_embeddings")

result3.head()

pytorch_model.bin:   6%|5         | 31.5M/547M [00:00<?, ?B/s]

/root/.cache/huggingface/modules/transformers_modules/nomic-ai/nomic-bert-2048/40b98394640e630d5276807046089b233113aa87/modeling_hf_nomic_bert.py:108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this 

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

CPU times: user 2min 11s, sys: 3.14 s, total: 2min 14s
Wall time: 2min 26s


,opinion_id,opinion_word_count,opinion,relevant_query_stmt,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",contractual obligations in maritime law,"[[0.00016998, -0.032441657, -0.018235508, -0.0...","[[0.032978956, -0.0062502497, -0.0033082487, -...",False,0.0,"[444, 293, 381, 381, 381]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,negligent retention employee sexual harassment,"[[0.05557652, -0.009500423, -0.005726365, 0.00...","[[0.04864664, 0.0039858404, -0.009498379, -0.0...",True,1.0,"[1, 1, 1, 112, 299]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,ineffective assistance of appellate counsel in...,"[[0.035781, 0.004495853, -0.01072409, -0.04589...","[[0.033952706, -0.002902296, -0.00263778, -0.0...",False,0.0,"[425, 218, 165, 218, 342]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,Roger Fincher's appeal of order denying rule 3...,"[[0.046108775, 0.035040826, -0.013530596, -0.0...","[[0.002662752, -0.01739174, -0.023292677, -0.0...",True,1.0,"[3, 359, 154, 365, 301]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,Appeal from a conviction for aggravated sexual...,"[[0.05488015, 0.01027694, -0.014723806, -0.034...","[[0.024143236, -0.005396211, -0.012538964, -0....",False,0.0,"[178, 20, 281, 11, 297]",4


In [15]:
result3.to_csv("4b.eval/IR-512/3.result3.csv", index=False)
len(result3)

450

In [16]:
EVALS

{'Alibaba-NLP/gte-large-en-v1.5': {'hit_rate': 0.8022222222222222,
  'mrr': 0.7117407407407407},
 'thenlper/gte-large': {'hit_rate': 0.8088888888888889,
  'mrr': 0.6997037037037037},
 'nomic-ai/nomic-embed-text-v1': {'hit_rate': 0.82, 'mrr': 0.7237777777777777}}

# Evaluate model: BAAI/bge-m3

In [17]:
%%time

model_name = 'BAAI/bge-m3'
model = SentenceTransformer('BAAI/bge-m3', trust_remote_code=True)
model.name = model_name
result4 = evaluate_model(df, model, "relevant_query_stmt", "query_embeddings", "opinion", "opinion_embeddings")

result4.head()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

CPU times: user 5min 20s, sys: 6.54 s, total: 5min 26s
Wall time: 6min 14s


,opinion_id,opinion_word_count,opinion,relevant_query_stmt,query_embeddings,opinion_embeddings,is_hit,mrr,retrieved,expected
0,8122327,241,"Benedict, D. J. Upon the testimony there- is ...",contractual obligations in maritime law,"[[-0.07329471, -0.0040467046, -0.0013884868, -...","[[-0.038305905, 0.032566477, -0.026009643, -0....",True,0.5,"[444, 0, 293, 448, 90]",0
1,2429336,1021,154 F. Supp. 2d 906 (2000) Margaret L. SUTPHIN...,negligent retention employee sexual harassment,"[[-0.062389757, 0.04123174, -0.022920785, -0.0...","[[-0.049252104, 0.0102680875, -0.044354577, -0...",True,1.0,"[1, 1, 1, 299, 299]",1
2,2939558,320,IN THE COURT OF CRIMINAL APPEALS OF TEXAS NO. ...,ineffective assistance of appellate counsel in...,"[[-0.026289232, 0.05550987, -0.017038338, -0.0...","[[-0.048325725, 0.011996219, -0.028630974, -0....",False,0.0,"[165, 425, 165, 92, 233]",2
3,4666766,84,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,Roger Fincher's appeal of order denying rule 3...,"[[-0.036555536, -0.0013679876, -0.022743266, -...","[[-0.05142785, -0.0016678479, -0.016272588, -0...",True,1.0,"[3, 428, 280, 195, 359]",3
4,1359066,4985,703 P.2d 699 (1985) 108 Idaho 935 Marcilena SH...,Appeal from a conviction for aggravated sexual...,"[[-0.037970044, 0.0041075842, -0.0013144802, 6...","[[-0.03737013, 0.013740901, -0.025323205, -0.0...",False,0.0,"[178, 20, 11, 232, 126]",4


In [18]:
result4.to_csv("4b.eval/IR-512/3.result4.csv", index=False)
len(result4)

450

In [19]:
EVALS

{'Alibaba-NLP/gte-large-en-v1.5': {'hit_rate': 0.8022222222222222,
  'mrr': 0.7117407407407407},
 'thenlper/gte-large': {'hit_rate': 0.8088888888888889,
  'mrr': 0.6997037037037037},
 'nomic-ai/nomic-embed-text-v1': {'hit_rate': 0.82, 'mrr': 0.7237777777777777},
 'BAAI/bge-m3': {'hit_rate': 0.7866666666666666, 'mrr': 0.6984814814814816}}